### Notebook 04: Model Tuning and Hybrid Models

This notebook tunes the baseline machine learning models (Logistic Regresssion, Random Forest, and XGBoost) and explores hybrid modeling approaches to improve prediction accuracy. Optuna is used for hyperparameter tuning.

In [26]:
import os
import numpy as np
import pandas as pd 
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Load preprocessed data
preprocessed_file = r'C:\Users\anitr\AAI590_Capstone\AAI590_Capstone_AH\Data\ea_modelset\eamodelset\dataset\preprocessed_models.csv'
models_df = pd.read_csv(preprocessed_file)
models_df.head()

,id,name,duplicateCount,elementCount,relationshipCount,viewCount,num_formats,hasWarning,hasDuplicate,rel_elem_ratio,view_elem_ratio,source_GitHub,source_Other,source_Unknown,language_bs,language_ca,language_cs,language_da,language_de,language_en,language_es,language_fi,language_fr,language_hr,language_id,language_it,language_ko,language_ms,language_nb,language_nl,language_nn,language_pl,language_pt,language_ru,language_sl,language_sv,language_tl,language_vi,language_yo,language_zh,arb_outcome
0,id-48fb3807bfa249a9bae607b6a92cc390,LAE,0,142,296,24,4,0,0,2.084507,0.169014,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4cc127d7-6937-42e8-99fb-19f0f6f4991a,Baseline Media Production,0,22,28,1,4,0,0,1.272727,0.045455,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,_7RWQ8CqVEey-A40W5C_9dw,buhService,0,55,41,3,4,0,0,0.745455,0.054545,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,3846c562-eab4-4e07-aa95-87703e0e0e69,Data model test,0,15,11,1,2,0,0,0.733333,0.066667,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,_ay028PGjEeqygJczXaaxEQ,payments-arch,0,18,20,1,4,0,0,1.111111,0.055556,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [6]:
# Define arb_outcome as target variable (y) and features (X)
X = models_df.drop(columns=['name','id','arb_outcome'])
y = models_df['arb_outcome']

# Check shape of X and y
print(f"Models DataFrame shape: {models_df.shape}")
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

Models DataFrame shape: (978, 41)
Features shape: (978, 38)
Target shape: (978,)


In [7]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Test set shape: {X_test.shape}, {y_test.shape}")

Training set shape: (782, 38), (782,)
Test set shape: (196, 38), (196,)


### XGBOOST - HYPERPARAMETER TUNING WITH OPTUNA

In [11]:
# Use optuna to optimize hyperparameters for XGBoost
def xgb_objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'objective': 'multi:softprob',
        'num_class': 3,
        'random_state': 42,
        'n_jobs': -1
    }
    
    xgb = XGBClassifier(**param)
    score = cross_val_score(xgb, X_train, y_train, cv=3, scoring='f1_macro', n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(xgb_objective, n_trials=25, show_progress_bar=True)
print("Best hyperparameters: ", study.best_params)
print("Best F1 Macro Score: ", study.best_value)



[I 2025-12-07 13:51:51,612] A new study created in memory with name: no-name-acd4a802-87ef-4951-829d-53443ea8a024
Best trial: 0. Best value: 0.981955:   4%|▍         | 1/25 [00:02<00:54,  2.27s/it]

[I 2025-12-07 13:51:53,879] Trial 0 finished with value: 0.9819554682423414 and parameters: {'n_estimators': 225, 'learning_rate': 0.10127098673703337, 'max_depth': 12, 'subsample': 0.945601409106067, 'colsample_bytree': 0.6965270370415978, 'gamma': 3.2108108771096373, 'min_child_weight': 3}. Best is trial 0 with value: 0.9819554682423414.


Best trial: 0. Best value: 0.981955:   8%|▊         | 2/25 [00:03<00:44,  1.93s/it]

[I 2025-12-07 13:51:55,578] Trial 1 finished with value: 0.9819554682423414 and parameters: {'n_estimators': 285, 'learning_rate': 0.21790987712319423, 'max_depth': 8, 'subsample': 0.9379372675096971, 'colsample_bytree': 0.8472522617357415, 'gamma': 2.2334149710812525, 'min_child_weight': 4}. Best is trial 0 with value: 0.9819554682423414.


Best trial: 0. Best value: 0.981955:  12%|█▏        | 3/25 [00:05<00:40,  1.84s/it]

...

[I 2025-12-07 13:52:04,645] Trial 19 finished with value: 0.9833862309464174 and parameters: {'n_estimators': 466, 'learning_rate': 0.14017890573961136, 'max_depth': 11, 'subsample': 0.8361829959634607, 'colsample_bytree': 0.6316418248059289, 'gamma': 1.9690633585746635, 'min_child_weight': 3}. Best is trial 11 with value: 0.989158277185438.
[I 2025-12-07 13:52:04,760] Trial 20 finished with value: 0.9719437643767158 and parameters: {'n_estimators': 339, 'learning_rate': 0.20635592111862402, 'max_depth': 9, 'subsample': 0.7787321154715885, 'colsample_bytree': 0.8882443783062456, 'gamma': 1.019350105766834, 'min_child_weight': 7}. Best is trial 11 with value: 0.989158277185438.


Best trial: 11. Best value: 0.989158:  92%|█████████▏| 23/25 [00:13<00:00,  6.30it/s]

[I 2025-12-07 13:52:04,929] Trial 21 finished with value: 0.989158277185438 and parameters: {'n_estimators': 500, 'learning_rate': 0.2714450558896942, 'max_depth': 10, 'subsample': 0.8611719550978623, 'colsample_bytree': 0.7318947256396325, 'gamma': 0.9400172879496328, 'min_child_weight': 1}. Best is trial 11 with value: 0.989158277185438.
[I 2025-12-07 13:52:05,095] Trial 22 finished with value: 0.989158277185438 and parameters: {'n_estimators': 500, 'learning_rate': 0.28107936341763085, 'max_depth': 10, 'subsample': 0.8689609803690862, 'colsample_bytree': 0.7248089781768803, 'gamma': 1.4774384882764053, 'min_child_weight': 1}. Best is trial 11 with value: 0.989158277185438.


Best trial: 11. Best value: 0.989158: 100%|██████████| 25/25 [00:13<00:00,  1.81it/s]

[I 2025-12-07 13:52:05,252] Trial 23 finished with value: 0.9877591954470054 and parameters: {'n_estimators': 454, 'learning_rate': 0.2002842010673283, 'max_depth': 11, 'subsample': 0.9278722478845978, 'colsample_bytree': 0.6965277360632465, 'gamma': 1.5331551680748203, 'min_child_weight': 2}. Best is trial 11 with value: 0.989158277185438.
[I 2025-12-07 13:52:05,419] Trial 24 finished with value: 0.9863131381222979 and parameters: {'n_estimators': 485, 'learning_rate': 0.12958566035006258, 'max_depth': 9, 'subsample': 0.8222609133340955, 'colsample_bytree': 0.7914706632469731, 'gamma': 2.6885968499815083, 'min_child_weight': 1}. Best is trial 11 with value: 0.989158277185438.
Best hyperparameters:  {'n_estimators': 492, 'learning_rate': 0.2516756545286703, 'max_depth': 11, 'subsample': 0.8477864785313033, 'colsample_bytree': 0.7655106019974527, 'gamma': 1.276914562872955, 'min_child_weight': 1}
Best F1 Macro Score:  0.989158277185438


In [12]:
# Train  optimized XGBoost model
xgboost_tuned = XGBClassifier(
    random_state=42, 
    n_estimators=study.best_params['n_estimators'], 
    learning_rate=study.best_params['learning_rate'], 
    max_depth=study.best_params['max_depth'], 
    subsample=study.best_params['subsample'], 
    colsample_bytree=study.best_params['colsample_bytree'], 
    gamma=study.best_params['gamma'],
    min_child_weight=study.best_params['min_child_weight'],
    objective="multi:softprob", 
    num_class=3
)
xgboost_tuned.fit(X_train, y_train)
y_pred_tuned = xgboost_tuned.predict(X_test)

# Evaluate tuned XGBoost model
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
print(f"Tuned XGBoost Model Accuracy: {accuracy_tuned:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred_tuned))

Tuned XGBoost Model Accuracy: 0.9949
Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        50
           1       1.00      0.99      1.00       105
           2       1.00      1.00      1.00        41

    accuracy                           0.99       196
   macro avg       0.99      1.00      1.00       196
weighted avg       0.99      0.99      0.99       196



### RANDOM FOREST - HYPERPARAMETER TUNING WITH OPTUNA

In [28]:
# Use optuna to optimize hyperparameters for Random Forest
def rf_objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 600),
        'max_depth': trial.suggest_int('max_depth', 5, 40),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced', 'balanced_subsample']),
        'random_state': 42,
        'n_jobs': -1
    }
    
    rf = RandomForestClassifier(**param)
    score = cross_val_score(rf, X_train, y_train, cv=3, scoring='f1_macro', n_jobs=-1).mean()
    return score

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(rf_objective, n_trials=25, show_progress_bar=True)
print("Best hyperparameters for Random Forest: ", study_rf.best_params)
print("Best F1 Macro Score for Random Forest: ", study_rf.best_value)


[I 2025-12-09 01:02:22,500] A new study created in memory with name: no-name-6622ec08-e92a-487d-a943-95820816e44d


Best trial: 0. Best value: 0.927799:   4%|▍         | 1/25 [00:03<01:28,  3.69s/it]

[I 2025-12-09 01:02:26,209] Trial 0 finished with value: 0.9277994701601587 and parameters: {'n_estimators': 418, 'max_depth': 23, 'min_samples_split': 14, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'bootstrap': True, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.9277994701601587.


Best trial: 1. Best value: 0.954474:   8%|▊         | 2/25 [00:05<01:01,  2.69s/it]

[I 2025-12-09 01:02:28,200] Trial 1 finished with value: 0.9544739068815103 and parameters: {'n_estimators': 332, 'max_depth': 22, 'min_samples_split': 15, 'min_samples_leaf': 7, 'max_features': 'sqrt', 'bootstrap': False, 'class_weight': None}. Best is trial 1 with value: 0.9544739068815103.


...

[I 2025-12-09 01:02:47,435] Trial 22 finished with value: 0.9926668537044878 and parameters: {'n_estimators': 556, 'max_depth': 26, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': False, 'class_weight': 'balanced'}. Best is trial 10 with value: 0.9926668537044878.


Best trial: 10. Best value: 0.992667:  96%|█████████▌| 24/25 [00:25<00:00,  1.55it/s]

[I 2025-12-09 01:02:48,131] Trial 23 finished with value: 0.9926668537044878 and parameters: {'n_estimators': 561, 'max_depth': 26, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': False, 'class_weight': 'balanced'}. Best is trial 10 with value: 0.9926668537044878.


Best trial: 10. Best value: 0.992667: 100%|██████████| 25/25 [00:26<00:00,  1.05s/it]

[I 2025-12-09 01:02:48,810] Trial 24 finished with value: 0.9926668537044878 and parameters: {'n_estimators': 561, 'max_depth': 18, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': False, 'class_weight': 'balanced'}. Best is trial 10 with value: 0.9926668537044878.
Best hyperparameters for Random Forest:  {'n_estimators': 575, 'max_depth': 28, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': False, 'class_weight': 'balanced'}
Best F1 Macro Score for Random Forest:  0.9926668537044878


In [29]:
# Train optimized Random Forest model
rf_tuned = RandomForestClassifier(
    random_state=42, 
    n_estimators=study_rf.best_params['n_estimators'],
    max_depth=study_rf.best_params['max_depth'],
    min_samples_split=study_rf.best_params['min_samples_split'],
    min_samples_leaf=study_rf.best_params['min_samples_leaf'],
    max_features=study_rf.best_params['max_features'],
    bootstrap=study_rf.best_params['bootstrap'],
    class_weight=study_rf.best_params['class_weight']
)
rf_tuned.fit(X_train, y_train)
y_pred_rf_tuned = rf_tuned.predict(X_test)

# Evaluate tuned Random Forest model
accuracy_rf_tuned = accuracy_score(y_test, y_pred_rf_tuned)
print(f"Tuned Random Forest Model Accuracy: {accuracy_rf_tuned:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred_rf_tuned))

Tuned Random Forest Model Accuracy: 1.0000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00       105
           2       1.00      1.00      1.00        41

    accuracy                           1.00       196
   macro avg       1.00      1.00      1.00       196
weighted avg       1.00      1.00      1.00       196



#### After hyperparameter tuning, the XGBoost model achieved an accuracy of 0.995, while the Random Forest model achieved an accuracy of 1.0 on the test set. The XGBoost model showed strong performance with a high F1 score across all classes, indicating its effectiveness in predicting ARB outcomes, but no improvement. The Random Forest modeled performed perfectly.

#### Next, ensemble models were created using both soft voting and stacking classifiers to combine the strengths of individual models.

### HYBRID MODEL - SOFT VOTING CLASSIFIER

In [18]:
# Create a soft voting classifier as hybrid model
hybrid_model_softvoting = VotingClassifier(
    estimators=[
        ('xgboost', xgboost_tuned),
        ('random_forest', rf_tuned),
        ('logistic_regression', LogisticRegression(max_iter=2000))
    ],
    voting='soft'
)

#Train ensemble
hybrid_model_softvoting.fit(X_train, y_train)

# Evaluate hybrid model
y_pred_hybrid = hybrid_model_softvoting.predict(X_test)
accuracy_hybrid = accuracy_score(y_test, y_pred_hybrid)
print(f"Hybrid Model Soft Voting Accuracy: {accuracy_hybrid:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred_hybrid))
 
    

Hybrid Model Soft Voting Accuracy: 0.9949
Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        50
           1       1.00      0.99      1.00       105
           2       1.00      1.00      1.00        41

    accuracy                           0.99       196
   macro avg       0.99      1.00      1.00       196
weighted avg       0.99      0.99      0.99       196



### HYBRID MODEL - STACKING CLASSIFIER

In [21]:
# Create a stacking classifier as hybrid model
hybrid_model_stacking = StackingClassifier(
    estimators=[
        ('xgboost', xgboost_tuned),
        ('random_forest', rf_tuned),
    ],
    final_estimator=LogisticRegression(),
    stack_method='predict_proba',
    passthrough=False,
    n_jobs=-1
)

# Train ensemble
hybrid_model_stacking.fit(X_train, y_train)

# Evaluate hybrid model
y_pred_hybrid_stack = hybrid_model_stacking.predict(X_test)
accuracy_hybrid_stack = accuracy_score(y_test, y_pred_hybrid_stack)
print(f"Hybrid Model Stacking Accuracy: {accuracy_hybrid_stack:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred_hybrid_stack))


Hybrid Model Stacking Accuracy: 1.0000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00       105
           2       1.00      1.00      1.00        41

    accuracy                           1.00       196
   macro avg       1.00      1.00      1.00       196
weighted avg       1.00      1.00      1.00       196



#### The stacking classifier achieved an accuracy of 1.00 on the test set, similar to the tuned Random Forest model. The soft voting classifier achived an accuract of 0.995, similar to the tuned XGBoost model, however, it did not outperform the Random Forest model or stacking classifier, which achieved perfect accuracy.

In [22]:
# Create comparison table of model accuracies
performance_data = {
    'Model': ['XGBoost Tuned', 'Random Forest Tuned', 'Hybrid Soft Voting', 'Hybrid Stacking'],
    'Accuracy': [accuracy_tuned, accuracy_rf_tuned, accuracy_hybrid, accuracy_hybrid_stack]
}

performance_df = pd.DataFrame(performance_data)
print(performance_df)


                 Model  Accuracy
0        XGBoost Tuned  0.994898
1  Random Forest Tuned  1.000000
2   Hybrid Soft Voting  0.994898
3      Hybrid Stacking  1.000000


#### Although the hybrid stacking and random forest models both achieved perfect accuracy, the stacking classifier provides a more robust approach by leveraging multiple models. Random Forest, however, produces more stable predictions.

In [27]:
# Save tuned and hybrid models
model_dir = r'C:\Users\anitr\AAI590_Capstone\AAI590_Capstone_AH\Models\tuned_and_hybrid_models'
os.makedirs(model_dir, exist_ok=True)
joblib.dump(xgboost_tuned, os.path.join(model_dir, 'xgboost_tuned.pkl'))
joblib.dump(rf_tuned, os.path.join(model_dir, 'rf_tuned.pkl'))
joblib.dump(hybrid_model_softvoting, os.path.join(model_dir, 'hybrid_softvoting.pkl'))
joblib.dump(hybrid_model_stacking, os.path.join(model_dir, 'hybrid_stacking.pkl'))

print("Tuned and hybrid models saved successfully.")

Tuned and hybrid models saved successfully.
